In [74]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [75]:
train = pd.read_csv("train.csv")
leaderboard = pd.read_csv("leaderboard_dataset.csv")
test = pd.read_csv("test.csv")

In [76]:
y = train["VAR21"]
X = train.drop(["VAR21"], axis=1)

In [77]:
from sklearn.preprocessing import LabelEncoder

for c in X.columns:
    if X[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(X[c].values) + list(test[c].values) + list(leaderboard[c].values)) 
        X[c] = lbl.transform(list(X[c].values))
        test[c] = lbl.transform(list(test[c].values))
        leaderboard[c] = lbl.transform(list(leaderboard[c].values))

In [78]:
lbl = LabelEncoder() 
lbl.fit(list(y.values)) 
y = lbl.transform(list(y))

In [79]:
y

array([1, 0, 2, ..., 1, 1, 0])

In [80]:
import xgboost as xgb

xgb_params = {
    'n_trees': 1000, 
    'eta': 0.005,
    'max_depth': 6,
    'subsample': 0.93,
    'objective': 'multi:softprob',
    'eval_metric': 'mlogloss',
    'silent': 1,
    'num_class' :3
}

from sklearn.model_selection import train_test_split
from sklearn.metrics.scorer import make_scorer

dtrain = xgb.DMatrix(X.drop(["VAR1"], axis=1), y)
dtest = xgb.DMatrix(leaderboard.drop(["VAR1"], axis=1))

model = xgb.train(dict(xgb_params, silent=1), dtrain, num_boost_round=1500)


# check f2-score (to get higher score - increase num_boost_round in previous cell)

# make predictions and save results
y_preds = model.predict(dtest)

In [81]:
dtest = xgb.DMatrix(test.drop(["VAR1"], axis=1))
y_preds = model.predict(dtest)


In [85]:
y_preds

array([[0.14465994, 0.27062294, 0.58471715],
       [0.14366013, 0.06739056, 0.7889493 ],
       [0.33066738, 0.04553473, 0.6237979 ],
       ...,
       [0.12418017, 0.46994412, 0.40587568],
       [0.09716163, 0.38539362, 0.5174448 ],
       [0.09234582, 0.55839145, 0.34926274]], dtype=float32)

In [91]:
y_mod = y_preds
a = []


for i in y_mod:
    if i[0]>i[1] or i[0]>i[2]:
        a.append(0)
    elif i[1]>i[0] and i[1]>i[2]:
        a.append(1)
    else:
        a.append(2)
        
d = {'col1': test["VAR1"], 'col2': [int(i) for i in a]}
df = pd.DataFrame(data=d)
df["col2"][df["col2"] == 0] = "High"
df["col2"][df["col2"] == 1] = "Low"
df["col2"][df["col2"] == 2] = "Medium"
print(df["col2"].value_counts())

import shutil 

filename = "Quant404_IITGuwahati"
df.to_csv(filename+'.csv', index=False, header=False)
shutil.copyfile("Quant404_IITGuwahati_54.ipynb", filename+".ipynb")

Medium    6561
Low       2273
High      1166
Name: col2, dtype: int64


'Quant404_IITGuwahati.ipynb'

In [71]:
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model = GaussianNB()
from sklearn.ensemble import BaggingClassifier

from sklearn.model_selection import cross_val_score

bag = BaggingClassifier(model, n_estimators=1000)

print(cross_val_score(bag, X.drop(["VAR1"], axis=1).fillna(0), y))

bag.fit(X.drop(["VAR1"], axis=1).fillna(0), y)
preds = bag.predict(leaderboard.drop(["VAR1"], axis=1).fillna(0))

d = {'col1': leaderboard["VAR1"], 'col2': [int(i) for i in preds]}
df = pd.DataFrame(data=d)
df["col2"][df["col2"] == 0] = "High"
df["col2"][df["col2"] == 1] = "Low"
df["col2"][df["col2"] == 2] = "Medium"
print(df["col2"].value_counts())

import shutil 

filename = "Quant404_IITGuwahati_78"
df.to_csv(filename+'.csv', index=False, header=False)
shutil.copyfile("Quant404_IITGuwahati_54.ipynb", filename+".ipynb")

/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[0.49470619 0.52730963 0.51557399]
Medium    7488
Low       2003
High       509
Name: col2, dtype: int64


'Quant404_IITGuwahati_77.ipynb'

In [42]:
train["VAR21"].value_counts()/3.4

Medium    5344.705882
Low       3553.235294
High      1102.058824
Name: VAR21, dtype: float64

In [ ]:
import shutil 

filename = "Quant404_IITGuwahati_67"
df.to_csv(filename+'.csv', index=False, header=False)
shutil.copyfile("Quant404_IITGuwahati_54.ipynb", filename+".ipynb")